In [1]:
import numpy as np
import scipy.sparse as sp
import relpy as rp
import matplotlib.pyplot as plt
import pydotplus
from IPython.display import Image
import time

In [2]:
def draw(x):
    dotf = x.todot()
    g = pydotplus.graph_from_dot_data(dotf)
    return Image(g.create_png())

In [3]:
with open('sip.py', 'w') as f:
    script = rp.parse('./avail.txt')
    f.write(script)

include ./param_sharpe
include ./models.txt


In [4]:
from sip import *

In [5]:
_time = rp.Parameter('t')
gamma = bind_gamma(gamma_die, gamma_hang)
pxy_gamma = bind_pxy_gamma(pxy_gamma_die, pxy_gamma_hang)
pairup = ctmc_pairup(b, beta_m, c, d, delta1, delta2, delta_m, e, gamma, mu, phi, q, r, rou_a, rou_m)
pxy_pairup = ctmc_pxy_pairup(pxy_b, pxy_beta_m, pxy_c, pxy_d, pxy_delta1, pxy_delta2, pxy_delta_m, pxy_e, pxy_gamma, pxy_mu, pxy_phi, pxy_q, pxy_r, pxy_rou_a, pxy_rou_m)
appserver = ctmc_appserver(b, beta_m, d, delta1, delta2, delta_m, e, gamma, mu, q, r, rou_a, rou_m)
proxy = ctmc_proxy(pxy_b, pxy_beta_m, pxy_d, pxy_delta1, pxy_delta2, pxy_delta_m, pxy_e, pxy_gamma, pxy_mu, pxy_q, pxy_r, pxy_rou_a, pxy_rou_m)
midplane = ctmc_midplane(alpha_sp, c_mp, mu_mp, plambda_mp)
cooling = ctmc_cooling(alpha_sp, mu_2c, mu_c, plambda_c)
power = ctmc_power(alpha_sp, c_ps, mu_2ps, mu_ps, plambda_ps)
processor = ctmc_processor(alpha_sp, mu_cpu, plambda_cpu)
memory = ctmc_memory(alpha_sp, mu_mem, plambda_mem)
disk = ctmc_disk(alpha_sp, chi_hd, mu_2hd, mu_hd, plambda_hd)
base = ctmc_base(alpha_sp, mu_base, plambda_base)
OS = ctmc_OS(alpha_sp, b_os, beta_os, delta_os, mu_os, plambda_os)
nic = ctmc_nic(alpha_sp, mu_nic, plambda_nic)
switch = ctmc_switch(alpha_sp, mu_esw, plambda_esw)
f_esw = bind_f_esw(plambda_esw, switch)
f_nic = bind_f_nic(nic, plambda_nic)
u_esw = bind_u_esw(f_esw, switch)
u_nic = bind_u_nic(f_nic, nic)
network = ftree_network(_time, plambda_esw, plambda_nic)
CM = ftree_CM(cooling, midplane, power)
BLADE = ftree_BLADE(OS, base, disk, memory, processor)
CLUSTER = ftree_CLUSTER(BLADE, CM, appserver, nic, proxy, switch)


In [6]:
start = time.time()
x = CLUSTER.seval(env)
print("elapsed_time:{0} [sec]".format(time.time() - start))

elapsed_time:0.1391749382019043 [sec]


In [7]:
params = CLUSTER.paramlist()
print(len(params))

58


In [8]:
start = time.time()
ddx = CLUSTER.sderiv2(env, params[0], params[1])
print("elapsed_time:{0} [sec]".format(time.time() - start))
ddx

elapsed_time:1.3399717807769775 [sec]


4.217352743086264e-14

In [9]:
start = time.time()
dx = [CLUSTER.sderiv(env, x) for x in params]
print("elapsed_time:{0} [sec]".format(time.time() - start))

elapsed_time:23.25560188293457 [sec]


In [ ]:
start = time.time()
dxx = [[CLUSTER.sderiv2(env, x1, x2) for x1 in params] for x2 in params]
print("elapsed_time:{0} [sec]".format(time.time() - start))